<a href="https://colab.research.google.com/github/taro-cell/Bean-Leaf-Lesions-Classification/blob/main/LangChian%E3%81%A8RAG%E3%82%92%E4%BD%BF%E3%81%84AI%E3%82%A8%E3%83%BC%E3%82%B8%E3%82%A7%E3%83%B3%E3%83%88%E3%82%92%E9%96%8B%E7%99%BA%E3%81%97%E3%81%9F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**0: Config**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install --quiet bertopic
!pip install --quiet nltk
!pip install --quiet llama_index

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 141.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 kB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.

In [ ]:
#libraries
import pandas as pd
import numpy as np
import matplotlib
import csv
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from cuml.cluster import HDBSCAN
from cuml.manifold import UMAP
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech, OpenAI
import google.generativeai as genai
from google.colab import userdata

/usr/local/lib/python3.12/dist-packages/hdbscan/robust_single_linkage_.py:175: SyntaxWarning: invalid escape sequence '\{'
  $max \{ core_k(a), core_k(b), 1/\alpha d(a,b) \}$.


In [ ]:
#hugging face config
!pip install huggingface_hub --quiet
from huggingface_hub import login
from google.colab import userdata

HF_TOKEN = userdata.get('HUGGING_FACE_TOKEN')
login(token=HF_TOKEN)

#**1: LamgChainを使って、トピックモデリングの出力を評価するAIエージェントを開発**

**Prepare Dataset**

In [ ]:
#read data
data = pd.read_csv('/content/drive/MyDrive/RAG_Research/dataset/data_all_relevant_text_cleaned.csv')
data_original = data.copy()

#make sure 'date' is date data type
data['date'] = pd.to_datetime(data['date'])

#count the number of words in body
data['body_cleaned'] = data['body_cleaned'].astype(str)
data['num_words_body'] = data['body_cleaned'].apply(lambda x: len(x.split()))

#data analyzing only bodies
data = data.drop_duplicates(subset=['body_cleaned'])


#remove short bodies
data = data[data['num_words_body'] > 50]


#after 2019
data = data[data['date'] >= pd.to_datetime('2019-09-01')]
data = data.reset_index(drop=True)
print(len(data))

texts = data['body_cleaned'].to_list()
len(texts)

2936


2936

In [ ]:
#split dataset into two (threshhold = 200)
data_over_words = data[data['num_words_body'] > 200]
print(len(data_over_words))
data_under_words = data[data['num_words_body'] <= 200]
print(len(data_under_words))

2577
359


In [ ]:
from llama_index.core.node_parser import SentenceSplitter

#class to split text
text_splitter = SentenceSplitter(chunk_size = 200, chunk_overlap = 10)

#split long texts
input_text = data_over_words['body_cleaned'].to_list()
all_splits = [s for a in input_text for s in text_splitter.split_text(a)]
print(len(all_splits))

10676


In [ ]:
tokens = data_under_words['body_cleaned'].to_list()
tokens.extend(all_splits)
print(len(tokens))

11035


**Initialize BERTopic**

In [ ]:
from sklearn.cluster import KMeans
#encode
embedding_model = SentenceTransformer('all-mpnet-base-v2')
embeddings = embedding_model.encode(tokens, show_progress_bar=True)

def topic_modeling(embeddings, umap_n_neighbors, umap_n_components, kmeans_n_clusters, tokens):

  #dimensionality reduction (UMAP)
  umap_model = UMAP(n_neighbors=umap_n_neighbors,
                    n_components= umap_n_components,
                    min_dist=0.0,
                    metric='cosine',
                    random_state=41)

  #clustering
  hdbscan_model = KMeans(n_clusters = kmeans_n_clusters)

  #vectorizer_model
  vectorizer_model = CountVectorizer(stop_words="english")

  #create instance
  topic_model = BERTopic(
    # Pipeline models
    embedding_model=embedding_model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer_model,

    # Hyperparameters
    top_n_words=10,
    verbose=True,
    calculate_probabilities = True
  )

  #fit BERTopic model
  topics, probs = topic_model.fit_transform(tokens, embeddings)

  return topic_model.get_topic_info()

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/345 [00:00<?, ?it/s]

**LLM-based topic coherence score**

In [ ]:
!pip install pydantic langchain-openai --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
#-----Setup-----
import os
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.exceptions import OutputParserException

#Set OpenAI API key
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

#-----Define the Structured Output Schema-----
class CoherenceResult(BaseModel):
    """Schema for the LLM's structured coherence score output."""
    top10_words: List[str] = Field(..., description="The list of top 10 words of the topic cluster.")
    coherence_score: float = Field(..., description="The coherence score for the top 10 words")
    #reasoning: str = Field(..., description="A brief explanation for the topic coherence score")

def get_coherence_score(
        top_k_words: List[str],
        llm_model: str = "gpt-4o-mini",
        temperature: float = 0.1
        )-> CoherenceResult:

    """
    Description

    """
    #Setupt output parser
    output_parser = PydanticOutputParser(pydantic_object=CoherenceResult)

    #Create template
    prompt_template = """
    You are a helpful assistant evaluating the top words of a topic model output for a given topic.
    Please rate how related the following words are to each other on a scale from 1 to 3 ('1' = not very related, '2' = moderately related, '3' = very related).
    The topic modeling is based on news articles on the discharge of the Fukushima treated radioactive water.
    The news articles cover a huge range of topics. Articles can include environmental risk, health risk, human right violation, geopolitical issue and government response.
    Reply with a single number, indicating the overall appropriateness of the topic.

    {format_instructions}

    Here are the top {word_count} words of the topic model output:
    {top_k_words}
    Let's think step by step.
    """

    prompt = ChatPromptTemplate.from_template(
        template = prompt_template,
        output_parser = output_parser,
        partial_variables = {"format_instructions": output_parser.get_format_instructions()}
    )

    #Initialize the LLM
    model = ChatOpenAI(model = llm_model,
                       temperature = temperature,
                       max_retries = 3)

    #Create and run the chain
    chain = prompt | model | output_parser

    #Invoke the chain
    try:
        result = chain.invoke({
            "word_count": len(top_k_words),
            "top_k_words": top_k_words
        })
        return result

    except Exception as e:
        print(f"Error: {e}")
        return None

**Logic**

In [ ]:
umap_n_neighbors_list = [20, 15, 10, 5]
umap_n_components_list = [7, 5, 3]
kmeans_n_clusters_list = [70, 50, 10, 5]

rows = []

for umap_n_neighbors in umap_n_neighbors_list:
  for umap_n_components in umap_n_components_list:
    for kmeans_n_clusters in kmeans_n_clusters_list:
      topic_model = topic_modeling(embeddings, umap_n_neighbors, umap_n_components, kmeans_n_clusters, tokens)
      topic_model = topic_model.loc[topic_model['Topic'] != -1]

      openai_scores = []

      topics = topic_model['Representation'].tolist()
      for top_10_words in topics:
        for _ in range(3):
            results = get_coherence_score(top_10_words)
            openai_scores.append(results.coherence_score)
      puw_score = PUW(topics, topk = 10)
      irbo_score = irbo_manual(topics, p = 0.9, topk = 10)
      row_dict = {"embedding_model": "all-mpet-base-v2",
                  "umap_n_neighbors": f"{umap_n_neighbors}",
                  "umap_n_components": f"{umap_n_components}",
                  "kmeans_n_clusters": f"{kmeans_n_clusters}",
                  "OpenAI Score": np.mean(openai_scores),
                  "puw_score": puw_score,
                  "irbo_score": irbo_score}
      rows.append(row_dict)

output_file = "/content/drive/MyDrive/RAG_Research/dataset/01_df_hp_tuning_1.csv"
#output_file = "/content/drive/MyDrive/RAG_Research/dataset/01_df_hp_tuning_2.csv"
#output_file = "/content/drive/MyDrive/RAG_Research/dataset/01_df_hp_tuning_3.csv"
#output_file = "/content/drive/MyDrive/RAG_Research/dataset/01_df_hp_tuning_4.csv"

with open(output_file, "w", newline="") as csvfile:
    fieldnames = rows[0].keys()
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in rows:
        writer.writerow(row)

2025-10-02 23:19:35,464 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[2025-10-02 23:19:35.457] [CUML] [info] build_algo set to brute_force_knn because random_state is given


2025-10-02 23:19:37,080 - BERTopic - Dimensionality - Completed ✓
2025-10-02 23:19:37,081 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-02 23:19:37,236 - BERTopic - Cluster - Completed ✓
2025-10-02 23:19:37,242 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-02 23:19:38,048 - BERTopic - Representation - Completed ✓
2025-10-02 23:55:03,942 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[2025-10-02 23:55:03.937] [CUML] [info] build_algo set to brute_force_knn because random_state is given


2025-10-02 23:55:04,305 - BERTopic - Dimensionality - Completed ✓
2025-10-02 23:55:04,306 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-02 23:55:04,401 - BERTopic - Cluster - Completed ✓
2025-10-02 23:55:04,406 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-02 23:55:05,635 - BERTopic - Representation - Completed ✓
2025-10-03 00:25:48,455 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


...

2025-10-03 02:21:21,392 - BERTopic - Dimensionality - Completed ✓
2025-10-03 02:21:21,393 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-03 02:21:21,475 - BERTopic - Cluster - Completed ✓
2025-10-03 02:21:21,480 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-03 02:21:22,250 - BERTopic - Representation - Completed ✓
2025-10-03 02:47:53,211 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[2025-10-03 02:47:53.206] [CUML] [info] build_algo set to brute_force_knn because random_state is given


2025-10-03 02:47:53,542 - BERTopic - Dimensionality - Completed ✓
2025-10-03 02:47:53,543 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-03 02:47:53,566 - BERTopic - Cluster - Completed ✓
2025-10-03 02:47:53,571 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-03 02:47:54,381 - BERTopic - Representation - Completed ✓
2025-10-03 02:53:00,925 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm


[2025-10-03 02:53:00.920] [CUML] [info] build_algo set to brute_force_knn because random_state is given


2025-10-03 02:53:01,264 - BERTopic - Dimensionality - Completed ✓
2025-10-03 02:53:01,265 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-10-03 02:53:01,278 - BERTopic - Cluster - Completed ✓
2025-10-03 02:53:01,283 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-10-03 02:53:02,032 - BERTopic - Representation - Completed ✓


#**3: RAGを使い外部データをソースとするAIエージェントを開発**

## **Create Database for Each Country**

In [ ]:
#-----Setup-----
#read dataset
df_for_analysis_final = pd.read_csv('/content/drive/MyDrive/RAG_Research/dataset/04_df_for_analysis_final.csv')

#filter only target countries
df_for_analysis_final = df_for_analysis_final.loc[df_for_analysis_final['country'].isin(['China', 'Japan', 'US'])]

#convert to date type
df_for_analysis_final['date'] = pd.to_datetime(df_for_analysis_final['date'])

#sort data by date
df_for_analysis_final.sort_values(by='date', inplace=True)

#-----Create a column of the most salient emotion-----
df_for_analysis_final.rename(columns={'label': 'most_salient_emotion'}, inplace=True)

#shape of df
print(df_for_analysis_final.shape)

#frequency of country
print(df_for_analysis_final['country'].value_counts())

#frequency of the most salient agenda attribute
print(df_for_analysis_final['most_salient_substantive_agenda_attribute'].value_counts())

#frequency of the most salient emotion
print(df_for_analysis_final['most_salient_emotion'].value_counts())

#create week number column
days_since_start = df_for_analysis_final['date'] - df_for_analysis_final['date'].min()
df_for_analysis_final['week_number'] = (days_since_start.dt.days // 7) + 1

df_for_analysis_final.head()

(2041, 27)
country
Japan    877
China    688
US       476
Name: count, dtype: int64
most_salient_substantive_agenda_attribute
Water Discharge Policy and Safety Validation                        992
Public Concerns                                                     579
Geopolitical and International Relations                            261
Incident in Fukushima Daiichi Nuclear Disaster and its Aftermath    176
Miscellaneous                                                        33
Name: count, dtype: int64
most_salient_emotion
neutral     951
fear        417
anger       341
disgust     199
sadness      84
joy          43
surprise      6
Name: count, dtype: int64


,title,body,company_name,country,western,date,body_cleaned,chunks,most_salient_topic,Representation,Representative_Docs,interpretation_1_openai,most_salient_substantive_agenda_attribute,pred,most_salient_emotion,score,anger,disgust,fear,joy,neutral,sadness,surprise,negative_3cls,neutral_3cls,positive_3cls,sentiment_label,week_number
2267,Nuclear Safety in China,Editor's Note: The State Council Information O...,China Daily,China,non-western,2019-09-04,Editor's Note: The State Council Information O...,Editor's Note: The State Council Information O...,prob_topic_29,"['nuclear', 'safety', 'development', 'grossi',...","['Established in 1984, the National Nuclear Sa...",Nuclear Safety and Development in China,Geopolitical and International Relations,4,neutral,0.790264,0.042718,0.015900,0.060465,0.066624,0.790264,0.010114,0.013915,0.000351,0.998857,0.000791,neutral_3cls,1
594,BRIEFING SESSION ON THE CURRENT STATUS OF MULT...,The following information was released by the ...,States News Service,US,western,2019-09-04,"1. On Wednesday, September 4, the Government o...","1. On Wednesday, September 4, the Government o...",prob_topic_10,"['alps', 'treated', 'iaea', 'review', 'fdnps',...","[""On 21 and 22 February, IAEA officials and in...",ALPS Treated Water Safety Review at Fukushima,Water Discharge Policy and Safety Validation,4,neutral,0.823646,0.024694,0.027528,0.016685,0.092184,0.823646,0.009309,0.005954,0.000348,0.998880,0.000771,neutral_3cls,1
2264,North Korea lambasts Japan over Fukushima,slammed a Japanese plan to discharge highly ra...,UPI,US,western,2019-09-05,slammed a Japanese plan to discharge highly ra...,slammed a Japanese plan to discharge highly ra...,prob_topic_15,"['water', 'plant', 'processing', 'liquid', 'al...","[""The Japanese government decided Tuesday to r...",Fukushima Treated Water Release,Water Discharge Policy and Safety Validation,0,anger,0.958319,0.958319,0.022334,0.010909,0.000562,0.003538,0.002626,0.001713,0.997629,0.002104,0.000266,negative_3cls,1
1745,Japan briefs diplomats on Fukushima nuclear wa...,TOKYO (AP) - Japan tried to reassure foreign d...,The Associated Press,US,western,2019-09-05,Japan tried to reassure foreign diplomats Wedn...,Japan tried to reassure foreign diplomats Wedn...,prob_topic_5,"['water', 'plant', 'tanks', 'release', 'power'...","[""Japan's government announced Tuesday it woul...",Fukushima Water Release,Water Discharge Policy and Safety Validation,4,neutral,0.571258,0.055475,0.099549,0.231118,0.011187,0.571258,0.025289,0.006125,0.002162,0.996914,0.000924,neutral_3cls,1
2266,Nuclear Safety in China,Editor's Note: The State Council Information O...,China Daily,China,non-western,2019-09-09,Editor's Note: The State Council Information O...,Editor's Note: The State Council Information O...,prob_topic_29,"['nuclear', 'safety', 'development', 'grossi',...","['Established in 1984, the National Nuclear Sa...",Nuclear Safety and Development in China,Geopolitical and International Relations,4,neutral,0.790264,0.042718,0.015900,0.060465,0.066624,0.790264,0.010114,0.013915,0.000351,0.998857,0.000791,neutral_3cls,1


In [ ]:
import os

#iterate country
for country in df_for_analysis_final['country'].unique():
  # Create country directory if it doesn't exist
  country_path = f"/content/drive/MyDrive/RAG_Research/RAG_Database_Weekly_ver4.0/{country}"
  if not os.path.exists(country_path):
    os.makedirs(country_path)


  #iterate each week
  for week in df_for_analysis_final['week_number'].unique():
    #create week directory inside country & agenda attribute directory if it does not exist
    week_path = f"{country_path}/{week}"
    if not os.path.exists(week_path):
        os.makedirs(week_path)

    #extract country and week number
    df_temp = df_for_analysis_final.loc[(df_for_analysis_final['country'] == country) & (df_for_analysis_final['week_number'] == week)]

    #text saved to the working directory
    txt_temp = df_temp['body_cleaned'].tolist()

    for i, txt in enumerate(txt_temp):
        with open(f"{week_path}/txt_{i}.txt", "w") as f:
            f.write(txt)

## **Build AI Agent for Each Country and Week**

In [ ]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
%pip install -qU "langchain[openai]"
%pip install -qU langchain-openai
%pip install -qU langchain-core
%pip install -qU langchain-community
%pip install -qU unstructured

In [ ]:
import getpass
import os
from google.colab import userdata

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import DirectoryLoader
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

#-----list of prompts-----
prompt_list = [{"Q1": "How do you evaluate the Japanese government's decision on the discharge of fukushima treated radioactive water?"},
               {"Q2": "What is the overall assessment of the Japanese government's decision to discharge the treated water from Fukushima?"},
               {"Q3": "Is the discharge of the treated water framed as a necessary and unavoidable step, or as a reckless and irresponsible action?"},
               {"Q4": "How is the scientific data regarding the safety of the treated water being presented?"},
               {"Q5": "What is the perspective on the Japanese government's and TEPCO's official safety assessments? Are they considered reliable and comprehensive?"},
               {"Q6": "Describe the moral and ethical considerations highlighted in the discussion of the Fukushima water release."},
               {"Q7": "How is Japan's transparency and communication with the international community portrayed?"},
               {"Q8": "What is the general assessment of the IAEA's credibility and competence regarding its review of Japan's water discharge plan?"},
               {"Q9": "Summarize the level of confidence expressed in the IAEA's ability to conduct a thorough and unbiased safety evaluation."},
               {"Q10": "Are there any discussions or accusations regarding the IAEA's potential lack of independence in this matter?"}]

#------Load documents into the database and Store the response in the dataframe-----
#country
for country in df_for_analysis_final['country'].unique():

    #placeholder of rows
    rows = []

    #config
    if not os.environ.get("OPENAI_API_KEY"):
        os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')


    df_target = df_for_analysis_final.loc[df_for_analysis_final['country'] == country]

    #week
    for week in df_target['week_number'].unique():
        row_dict = {"week": week}

        #define embedding model
        embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

        #define vector database
        vector_store = InMemoryVectorStore(embeddings)

        #-----Load documents to vector database-----
        loader = DirectoryLoader(f"/content/drive/MyDrive/RAG_Research/RAG_Database_Weekly_ver4.0/{country}/{week}", glob="**/*.txt")
        docs = loader.load()

        #-----Split long documents into small chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap = 200)
        all_splits = text_splitter.split_documents(docs)
        print(len(all_splits))

        #indexing
        _ = vector_store.add_documents(documents= all_splits)

        #pull template of query
        rag_prompt = hub.pull("rlm/rag-prompt")

        #define data structure
        class State(TypedDict):
            question: str
            context: List[Document]
            answer: str

        #define retrieve function
        def retrieve(state: State):
            retreieved_docs = vector_store.similarity_search(state["question"], k = 5)
            return {"context": retreieved_docs}

        #define generation function
        def generate(state: State):
            docs_content = "\n\n".join(doc.page_content for doc in state["context"])
            messages = rag_prompt.invoke({"question": state["question"], "context": docs_content})
            response = llm.invoke(messages)
            return {"answer": response.content}

        #initialize graph builder
        graph_builder = StateGraph(State).add_sequence([retrieve, generate])
        graph_builder.add_edge(START, "retrieve")
        graph = graph_builder.compile()

        #-----Invoke llm agent-----
        for current_prompt in prompt_list:
            col_name = list(current_prompt.keys())[0]
            response = graph.invoke({"question": list(current_prompt.values())[0]})

            #append the dict
            row_dict.update({col_name: response['answer']})
        rows.append(row_dict)

    df_temp = pd.DataFrame(rows)
    df_temp.to_csv(f"/content/drive/MyDrive/RAG_Research/RAG_Database_Weekly_ver4.0/{country}/weekly_summary.csv", index = False)

df_temp.head()

10
3
16
2
7
2
7
6
46
15
3
14
8
6
14
2
7
6
5
3
7
7
5
27
7
12
3
10
5
5
92
7
11
134
51
12
3
15
7
11
15
15
20
4
5
5
15
8
21
2
16
4
10
4
21
5
8
21
33
12
18
2
15
5
6
2
8
5
3
1
4
1
7
2
17
2
7
4
27
15
6
18
10
8
15
12
34
23
7
2
19
16
5
18
9
24
18
26
20
4
27
50
124
20
16
28
2
6
122
227
35
34
41
3
21
5
20
25
27
2
4
5
13
1
19
3
6
4
13
10
4
12
26
8
9
3
4
3
15
2
1
2
50
5
12
7
8
8
3
6
3
2
11
3
5
13
111
2
3
8
5
5
4
2
18
23
3
11
91
129
24
67
8
9
18
3
4
5
5
4
4
3
2
2
12
4
3
3
11
22
10
4
5
2
23
3
6
20
3
3
4
10
28
40
4
4
7
3
6
19
4
6
4
18
5
27
29
96
57
30
15
14
56
9
17
11
39
80
23
18
84
34
52
80
195
59
38
61
15
61
116
367
105
80
14
21
36
45
12
8
15
3
3
1
11
1
15
4
7
2
15
10
20
20
26
2
5
7
27
3
5
1
3
5
6
6
8
3
26
24
8
4
6
2
8
14
14
11
7
11
7
8
8
3
7
3
13
11
10
6
13
4
11
34
4
5
33
35
4
3
7
4
20
8
13
4
8
3
3
10
4
1
9
4
17
21
11
26
16
20
20
9
8
14
4
3
8
3
4
4
8
6
34
10
7
8
13
12
3
29
76
20
38
11
3
5
45
153
19
13
14
15
4
14
4
20
9
20
4
6
6
4
8
10
9
3
4
5
23
5
10
3
4
11
4
5
2
8
3
10
3
6
11
3
6
2
21
12
2
1
4
3
2

,week,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10
0,2,The evaluation of the Japanese government's de...,The overall assessment of the Japanese governm...,The discharge of the treated water is framed a...,The scientific data regarding the safety of th...,The Japanese government's and TEPCO's official...,The moral and ethical considerations in the di...,Japan's transparency and communication with th...,The general assessment of the IAEA's credibili...,The retrieved context does not clearly express...,The retrieved context does not provide specifi...
1,11,The Japanese government's decision to discharg...,The overall assessment of the Japanese governm...,The discharge of treated water is framed as a ...,The scientific data regarding the safety of th...,The perspective on the Japanese government and...,The moral and ethical considerations surroundi...,Japan's transparency and communication with th...,The general assessment of the IAEA's credibili...,The IAEA's ability to conduct a thorough and u...,The retrieved context does not explicitly ment...
2,12,The Japanese government's decision to discharg...,The overall assessment of the Japanese governm...,The discharge of treated water is framed as a ...,The Japanese government has presented scientif...,The perspective on the Japanese government's a...,The moral and ethical considerations surroundi...,Japan's transparency and communication with th...,The general assessment of the IAEA's credibili...,The context does not provide specific informat...,The context does not provide any specific disc...
3,16,The Japanese government's decision to discharg...,The Japanese government's decision to discharg...,The discharge of the treated water is framed a...,The scientific data regarding the safety of th...,The credibility of the Japanese government's a...,The discussion of the Fukushima water release ...,Japan's transparency and communication with th...,The credibility and competence of the IAEA reg...,The IAEA's level of confidence in conducting a...,The context does not specifically mention any ...
4,17,Evaluating the Japanese government's decision ...,The overall assessment of the Japanese governm...,The discharge of treated water is framed as a ...,The scientific data regarding the safety of tr...,The reliability and comprehensiveness of the J...,The moral and ethical considerations surroundi...,Japan's transparency and communication with th...,The IAEA's credibility and competence regardin...,The context does not explicitly express confid...,The retrieved context does not explicitly ment...
